# Nashville_History

In [ ]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

## Council Districts

In [ ]:
districts = gpd.read_file('../data/Council District Outlines (GIS).geojson')
print(districts.crs)
districts.head()

In [ ]:
districts = districts[['council_district', 'geometry']]

In [ ]:
#leg_districts = {'title':'Council Districts', 'loc':'upper left','bbox_to_anchor':(1,1.01),'ncol':2}

#districts.plot(column = 'council_district', figsize=(10, 10),
#               edgecolor = 'black',
#               legend = True, legend_kwds = leg_districts,
#               cmap = 'Set3')
#plt.show();

fig, ax = plt.subplots(figsize = (10,10))
districts.plot(ax = ax, 
               column = 'council_district', 
               cmap = 'Set3',
               edgecolor = 'black')
ax.axis('off')

for index, row in districts.iterrows():
    
    x=row['geometry'].centroid.x
    y=row['geometry'].centroid.y

    if row.council_district == '18':
        x=x #+ 0.02
        y=y - 0.005
    elif row.council_district == '19':
        x=x - 0.013
        y=y - 0.004
        
    plt.annotate(text=row['council_district'],
                 xy=(x, y),
                 horizontalalignment='center', fontweight='bold')

plt.title('Nashville Council Districts')
plt.show();

## Historical Markers

In [ ]:
hist_markers = gpd.read_file('../data/Historical Markers.geojson')
print(hist_markers.crs)
print(hist_markers.shape)
hist_markers.head()

## Historical Commission Preservation Awards

In [ ]:
hist_awards = gpd.read_file('../data/Historical Commission Preservation Awards.geojson')
print(hist_awards.crs)
print(hist_awards.shape)
hist_awards.head()

## Spatial Joins

In [ ]:
hist_markers_by_district = gpd.sjoin(hist_markers, districts, op = 'within')

hist_awards_by_district = gpd.sjoin(hist_awards, districts, op = 'within')

In [ ]:
hist_markers_by_district.council_district.value_counts()
#District 19 has 72 historical markers

hist_awards_by_district.council_district.value_counts()
#District 19 has 157 historical commission preservation awards

## District 19

In [ ]:
polygon_d19 = districts.loc[districts.council_district == '19']
polygon_d19.shape

In [ ]:
hist_markers_in_district19 = hist_markers_by_district.loc[hist_markers_by_district.council_district == '19']
print(hist_markers_in_district19.shape)
hist_markers_in_district19.head()

In [ ]:
hist_awards_in_district19 = hist_awards_by_district.loc[hist_awards_by_district.council_district == '19']
print(hist_awards_in_district19.shape)
hist_awards_in_district19.head()

In [ ]:
# Historical Markers in Council District 19

ax = polygon_d19.plot(figsize = (8, 10), color = 'lightgreen')
hist_markers_in_district19.plot(ax = ax)
plt.title('Historical Markers in Council District 19')
plt.show();

In [ ]:
# Historical Commision Preservation Awards in Council District 19

leg_awards = {'title':'Preservation Award Category', 'loc':'upper left','bbox_to_anchor':(1,1.01),'ncol':1}

az = polygon_d19.plot(figsize = (8, 10), color = 'lightgreen')
hist_awards_in_district19.plot(ax = az, column = 'category', legend = True, legend_kwds = leg_awards)
plt.title('Historical Commision Preservation Awards in Council District 19')
plt.show();

## Folium Base Map

In [ ]:
polygon_d19.geometry.centroid

In [ ]:
center = polygon_d19.geometry.centroid[18]
print(center)

In [ ]:
area_center = [center.y + 0.003, center.x]
print(area_center)

## Folium Maps with Markers

In [ ]:
# Historical Markers in Council District 19

#Draw the district
map_hmarkers_district19 = folium.Map(location = area_center, 
                                     zoom_start = 14,
                                     min_zoom = 12)

folium.GeoJson(polygon_d19).add_to(map_hmarkers_district19)

#Add title
title_hm_d19 = 'Historical Markers in Council District 19'
title_hm_d19_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(title_hm_d19)
map_hmarkers_district19.get_root().html.add_child(folium.Element(title_hm_d19_html))

#iterate through historical markers to create locations and markers for each historical marker.

for row_index, row_values in hist_markers_in_district19.iterrows():
    loc = [row_values['latitude'], row_values['longitude']] 
    pop = str(row_values['marker_name']) + '<br><br>' + 'Erected ' + str(row_values['year_erected'])
    icon= folium.Icon(color="blue",icon="info-circle", prefix='fa')
        
    marker = folium.Marker(
        location = loc,
        popup = pop, 
        icon = icon)
    
    marker.add_to(map_hmarkers_district19)
map_hmarkers_district19.save('../maps/map_histmarkers_district19.html')

map_hmarkers_district19

In [ ]:
# Historical Commision Preservation Awards in Council District 19

#Draw the district
map_hawards_district19 = folium.Map(location = area_center, 
                                    zoom_start = 14,
                                    min_zoom = 12)

folium.GeoJson(polygon_d19).add_to(map_hawards_district19)

#add title
title_ha_d19 = 'Historical Commission Preservation Awards in Council District 19'
title_ha_d19_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(title_ha_d19)
map_hawards_district19.get_root().html.add_child(folium.Element(title_ha_d19_html))

#iterate through preservation awards to create locations and markers for each preservation award.

for row_index, row_values in hist_awards_in_district19.iterrows():
    loc = [row_values.geometry.y, row_values.geometry.x]
    pop = str(row_values['building_name'])+'<br><br>'+str(row_values['category']+',<br>Awarded '+ str(row_values['year']))
    
    #icon= folium.Icon(color="darkblue",icon="home", prefix='fa')
    
    if row_values.category == 'Commercial':
        icon= folium.Icon(color="blue",icon="building", prefix='fa')
    elif row_values.category == 'Educational/Institutional':
        icon= folium.Icon(color="green",icon="university", prefix='fa') 
    elif row_values.category == 'Engineering/Industrial':
        icon= folium.Icon(color="gray",icon="industry", prefix='fa')    
    elif row_values.category == 'Infill':
        icon= folium.Icon(color="darkgreen",icon="building-o", prefix='fa')    
    elif row_values.category == 'Mixed Use':
        icon= folium.Icon(color="purple",icon="shopping-bag", prefix='fa')    
    elif row_values.category == 'Neighborhood':
        icon= folium.Icon(color="darkpurple",icon="users", prefix='fa') 
    elif row_values.category == 'Public Buildings':
        icon= folium.Icon(color="darkblue",icon="institution", prefix='fa')  
    elif row_values.category == 'Religious':
        icon= folium.Icon(color="orange",icon="universal-access", prefix='fa')
    else: #row_values.category == 'Residential':
        icon= folium.Icon(color="lightgray",icon="home", prefix='fa')  
        
    marker = folium.Marker(
        location = loc,
        popup = pop, 
        icon = icon)
    
    marker.add_to(map_hawards_district19)
map_hawards_district19.save('../maps/map_histawards_district19.html')

map_hawards_district19

In [ ]:
#folium.Map?

## Marker Cluster

In [ ]:
# Historical Markers & Historical Commission Preservation Awards in Council District 19

cluster_map_district19 = folium.Map(location = area_center, 
                                    zoom_start = 14,
                                    min_zoom = 12,
                                    #crs='EPSG4326',  #This moved the base map to Norway???
                                    tiles='OpenStreetMap'  #default -- none of the others work
                                    #tiles='Stamen'
                                    #tiles='Mapbox Bright'
                                    #tiles='Mapbox Control Room'
                                   )

marker_cluster = MarkerCluster().add_to(cluster_map_district19)

folium.GeoJson(polygon_d19).add_to(cluster_map_district19)

#add title
title_hmha_d19 = 'Historical Markers and Preservation Awards in Council District 19'
title_hmha_d19_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format(title_hmha_d19)
cluster_map_district19.get_root().html.add_child(folium.Element(title_hmha_d19_html))

for row_index, row_values in hist_markers_in_district19.iterrows():
    loc = [row_values.latitude, row_values.longitude]
    pop = str(row_values['marker_name']) + '<br><br>' + 'Erected ' + str(row_values['year_erected'])
    icon= folium.Icon(color="blue", icon="info-circle", prefix='fa')
    
    marker = folium.Marker(
        location = loc,
        popup = pop,
        icon = icon)
    
    marker.add_to(marker_cluster)

for row_index, row_values in hist_awards_in_district19.iterrows():
    loc = [row_values.geometry.y, row_values.geometry.x]
    pop = str(row_values['building_name'])+'<br><br>'+str(row_values['category']+',<br>Awarded '+ str(row_values['year']))
    #icon= folium.Icon(color="darkblue", icon="home", prefix='fa')

    if row_values.category == 'Commercial':
        icon= folium.Icon(color="darkblue",icon="building", prefix='fa')
    elif row_values.category == 'Educational/Institutional':
        icon= folium.Icon(color="darkblue",icon="university", prefix='fa')
    elif row_values.category == 'Engineering/Industrial':
        icon= folium.Icon(color="darkblue",icon="industry", prefix='fa')    
    elif row_values.category == 'Infill':
        icon= folium.Icon(color="darkblue",icon="building-o", prefix='fa')    
    elif row_values.category == 'Mixed Use':
        icon= folium.Icon(color="darkblue",icon="shopping-bag", prefix='fa')    
    elif row_values.category == 'Neighborhood':
        icon= folium.Icon(color="darkblue",icon="users", prefix='fa')    
    elif row_values.category == 'Public Buildings':
        icon= folium.Icon(color="darkblue",icon="institution", prefix='fa')
    elif row_values.category == 'Religious':
        icon= folium.Icon(color="darkblue",icon="universal-access", prefix='fa')
    else: #row_values.category == 'Residential':
        icon= folium.Icon(color="darkblue",icon="home", prefix='fa')
    
    marker = folium.Marker(
        location = loc,
        popup = pop,
        icon = icon)
    
    marker.add_to(marker_cluster)

    
cluster_map_district19.save('../maps/cluster_district12.html')

cluster_map_district19